In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
from IPython.display import display, Markdown

import sys
sys.path.append('../')

from evalutils.roc import get_bootstrapped_roc_ci_curves
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from utilities import data, roc

## directory where results are
EXPERIMENT_DIR = f"/data/bodyct/experiments/lung-malignancy-fairness-shaurya"
NLST_PREDS = f"{EXPERIMENT_DIR}/nlst"

TEAMS_DIR = "C:/Users/shaur/OneDrive - Radboudumc/Documents - Master - Shaurya Gaur/General/Malignancy-Estimation Results"
NLST_PREDS = f"{TEAMS_DIR}/nlst" ## Comment out if not using Teams backup (aka Chansey is up :)
RESULTS_DIR = f"{TEAMS_DIR}/fairness-analysis-results"

## DLCST

In [2]:
dlcst_preds = pd.read_csv(f"{TEAMS_DIR}/dlcst/dlcst_thijmen_kiran_sybil_malignancy_estimation_results.csv", header=0)
dlcst_preds

,PatientID,StudyDate,SeriesInstanceUID,Age,Sex,FamilyHistoryLungCa,Emphysema,NoduleCountPerScan,sybil_year1,sybil_year2,sybil_year3,sybil_year4,sybil_year5,sybil_year6,PanCan2b,Ensemble_Kiran,thijmen_mean,label
0,4,20050124,1.2.840.113704.1.111.4964.1106577805.10,55,2,0,0,9,0.021629,0.038573,0.071919,0.079270,0.095846,0.135681,0.053366,0.082652,0.166209,0
1,35,20051208,1.2.840.113704.1.111.5776.1134059140.11,56,1,1,1,2,0.001170,0.002554,0.007835,0.011039,0.018442,0.030460,0.009543,0.000408,0.003368,0
2,38,20060109,1.2.840.113704.1.111.2004.1136823831.14,62,2,0,1,4,0.001784,0.003870,0.007835,0.012797,0.019229,0.032957,0.006734,0.002702,0.065888,0
3,47,20051214,1.2.840.113704.1.111.8148.1134579622.14,57,1,0,1,1,0.003951,0.015674,0.025373,0.034010,0.040605,0.058852,0.007944,0.084158,0.423341,0
4,56,20051213,1.2.840.113704.1.111.2744.1134487263.11,64,1,0,1,3,0.000000,0.001574,0.003791,0.006847,0.010381,0.017287,0.000899,0.000013,0.005590,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,4057,20060314,1.2.840.113704.1.111.4796.1142355218.14,69,1,0,1,2,0.168810,0.273494,0.257961,0.294720,0.327062,0.383196,0.380198,0.893933,0.682322,0
595,4063,20060313,1.2.840.113704.1.111.5104.1142267340.10,55,1,0,1,4,0.001965,0.006793,0.013618,0.017289,0.021685,0.035951,0.000000,0.000099,0.103020,0
596,4079,20060328,1.2.840.113704.1.111.1308.1143556124.11,52,2,0,0,1,0.003951,0.011459,0.025373,0.034010,0.040101,0.058852,0.039054,0.070744,0.121373,0
597,4098,20060403,1.2.840.113704.1.111.5848.1144079789.11,54,1,0,1,3,0.117795,0.167839,0.189976,0.217799,0.229753,0.300137,0.134158,0.452513,0.347016,0


In [3]:
dlcst_democols = {
    'cat': {'demo': ['Sex'], 'other': ['FamilyHistoryLungCa', 'Emphysema']},
    'num': {'demo': ['Age'], 'other': ['NoduleCountPerScan']}
}

In [4]:
DLCST_MODELCOLS = {
    "Venkadesh": "Ensemble_Kiran",
    "de Haas": "thijmen_mean",
    "PanCan2b": "PanCan2b",
    "Sybil year 1": "sybil_year1",
    # "Sybil year 2": "sybil_year2",
    # "Sybil year 3": "sybil_year3",
    # "Sybil year 4": "sybil_year4",
    # "Sybil year 5": "sybil_year5",
    # "Sybil year 6": "sybil_year6",
}

In [5]:
result_df = roc.all_results_subgroups_models(dlcst_preds, dlcst_democols['cat'], models=DLCST_MODELCOLS, csvpath=f"{RESULTS_DIR}/auroc-dlcst-{len(dlcst_preds)}.csv")
result_df

,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.437917,0.775716,1.0,0.908254,0.837521,0.970516,2.0,0.944307,0.908594,0.968446,...,291,53.923205,9.907121,27,249,46.076795,9.782609,Ensemble_Kiran,Sex,demo
de Haas,0.353367,0.928078,1.0,0.902960,0.839925,0.965447,2.0,0.946357,0.913565,0.974769,...,291,53.923205,9.907121,27,249,46.076795,9.782609,thijmen_mean,Sex,demo
PanCan2b,0.350830,-0.932981,1.0,0.909613,0.851650,0.953015,2.0,0.856363,0.792547,0.900080,...,291,53.923205,9.907121,27,249,46.076795,9.782609,PanCan2b,Sex,demo
Sybil year 1,0.881130,-0.149537,1.0,0.866212,0.776390,0.946607,2.0,0.857134,0.782818,0.922584,...,291,53.923205,9.907121,27,249,46.076795,9.782609,sybil_year1,Sex,demo
Venkadesh,0.387424,0.864298,0.0,0.914361,0.851641,0.952758,1.0,0.954671,0.917565,0.986973,...,454,82.971619,8.651911,16,86,17.028381,15.686275,Ensemble_Kiran,FamilyHistoryLungCa,other
de Haas,0.189745,1.311334,0.0,0.909660,0.843920,0.953267,1.0,0.966651,0.930070,0.997565,...,454,82.971619,8.651911,16,86,17.028381,15.686275,thijmen_mean,FamilyHistoryLungCa,other
PanCan2b,0.824541,0.221708,0.0,0.884131,0.827016,0.922856,1.0,0.897716,0.829119,0.946948,...,454,82.971619,8.651911,16,86,17.028381,15.686275,PanCan2b,FamilyHistoryLungCa,other
Sybil year 1,0.760273,0.305122,0.0,0.870294,0.801313,0.934633,1.0,0.889633,0.782195,0.962835,...,454,82.971619,8.651911,16,86,17.028381,15.686275,sybil_year1,FamilyHistoryLungCa,other
Venkadesh,0.210925,1.251025,1.0,0.910710,0.853562,0.954101,0.0,0.965888,0.940037,0.988043,...,361,67.612688,10.864198,15,179,32.387312,7.731959,Ensemble_Kiran,Emphysema,other
de Haas,0.508438,0.661272,1.0,0.915596,0.866233,0.960734,0.0,0.948117,0.902625,0.978142,...,361,67.612688,10.864198,15,179,32.387312,7.731959,thijmen_mean,Emphysema,other


## NLST

In [6]:
nlst_preds_nodule = pd.read_csv(f"{NLST_PREDS}/nlst_demov4_allmodels_cal.csv")

with open(f'{NLST_PREDS}/nlst_demo_v4_cols.json') as json_data:
    nlst_democols = json.load(json_data)
    json_data.close()

nlst_democols['cat'].pop('lungcanc')
nlst_democols

{'num': {'demo': ['BMI', 'Age', 'height', 'weight'],
  'smoke': ['smokeage', 'smokeday', 'smokeyr', 'pkyr'],
  'nodule': ['CoordX', 'CoordZ', 'CoordY'],
  'other': ['NoduleCounts', 'Diameter [mm]']},
 'cat': {'demo': ['Overweight',
   'educat',
   'Gender',
   'Married',
   'HighSchoolPlus',
   'NonHispanicWhite',
   'Unfinished_ed',
   'WhiteOrBlack',
   'marital',
   'ethnic',
   'race'],
  'smoke': ['smokelive', 'cigar', 'cigsmok', 'smokework', 'pipe'],
  'work': ['wrkbaki',
   'wrkfoun',
   'wrkchem',
   'wrkasbe',
   'wrkfire',
   'wrksand',
   'wrkfarm',
   'wrkcoal',
   'wrkpain',
   'wrkweld',
   'wrkflou',
   'wrkbutc',
   'wrkhard',
   'wrkcott'],
  'disease': ['diagasbe',
   'diagchas',
   'diagpneu',
   'diagstro',
   'diagemph',
   'diagbron',
   'diagsili',
   'diagsarc',
   'diaghear',
   'diagdiab',
   'diagadas',
   'diagcopd',
   'diagfibr',
   'diagtube',
   'diaghype',
   'diagchro'],
  'canchist': ['canckidn',
   'cancphar',
   'canccolo',
   'cancoral',
   'cancpa

### Nodule-level

In [7]:
MODELS = {
    "Venkadesh": "DL_cal",
    # "de Haas Combined": "Thijmen_mean",
    "de Haas Local": "Thijmen_local_cal",
    "de Haas Global (hidden nodule)": "Thijmen_global_hidden_cal",
    "de Haas Global (shown nodule)": "Thijmen_global_show_cal",
    "PanCan2b": "PanCan2b",
    # "Sybil year 1": "sybil_year1",
    # "Sybil year 2": "sybil_year2",
    # "Sybil year 3": "sybil_year3",
    # "Sybil year 4": "sybil_year4",
    # "Sybil year 5": "sybil_year5",
    # "Sybil year 6": "sybil_year6",
}

nlst_preds = data.prep_nlst_preds(nlst_preds_nodule, scanlevel=False, tijmen=False, sybil=False)
print(len(nlst_preds), " nodules")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv")
result_df

16077  nodules


c:\Users\shaur\Documents\bodyct-lung-malignancy-fairness\utilities\roc.py:59: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(


,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.797927,-0.256031,True,0.909175,0.899512,0.919816,False,0.906173,0.89366,0.92017,...,10211,68.514026,7.299138,445,4617,31.485974,8.790992,DL_cal,Overweight,demo
de Haas Local,0.467947,0.725824,True,0.892398,0.881897,0.904742,False,0.901244,0.888641,0.914005,...,10211,68.514026,7.299138,445,4617,31.485974,8.790992,Thijmen_local_cal,Overweight,demo
de Haas Global (hidden nodule),0.932304,0.084947,True,0.862032,0.853848,0.872692,False,0.863193,0.848362,0.876154,...,10211,68.514026,7.299138,445,4617,31.485974,8.790992,Thijmen_global_hidden_cal,Overweight,demo
de Haas Global (shown nodule),0.049413,-1.965012,True,0.898451,0.888532,0.905758,False,0.873113,0.85809,0.888516,...,10211,68.514026,7.299138,445,4617,31.485974,8.790992,Thijmen_global_show_cal,Overweight,demo
PanCan2b,0.086159,-1.716014,True,0.844504,0.833357,0.85679,False,0.819162,0.799672,0.839936,...,10211,68.514026,7.299138,445,4617,31.485974,8.790992,PanCan2b,Overweight,demo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venkadesh,0.199439,1.283152,LightSpeed QX/i,0.895073,0.869483,0.915284,Volume Zoom,0.919862,0.89897,0.937618,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,DL_cal,ManufacturersModelName,scanner
de Haas Local,0.458104,0.741973,LightSpeed QX/i,0.87524,0.851591,0.900752,Volume Zoom,0.891072,0.865797,0.913374,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,Thijmen_local_cal,ManufacturersModelName,scanner
de Haas Global (hidden nodule),0.852734,0.185631,LightSpeed QX/i,0.863618,0.844484,0.883522,Volume Zoom,0.86781,0.848926,0.882675,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,Thijmen_global_hidden_cal,ManufacturersModelName,scanner
de Haas Global (shown nodule),0.744952,0.325303,LightSpeed QX/i,0.896691,0.873727,0.913598,Volume Zoom,0.903232,0.884644,0.922774,...,2041,14.194190,10.560911,181,3036,20.009952,5.626360,Thijmen_global_show_cal,ManufacturersModelName,scanner


In [8]:
MODELS = {
    "Venkadesh": "DL_cal",
    "de Haas Combined": "Thijmen_mean",
    "de Haas Local": "Thijmen_local_cal",
    "de Haas Global (hidden nodule)": "Thijmen_global_hidden_cal",
    "de Haas Global (shown nodule)": "Thijmen_global_show_cal",
    "PanCan2b": "PanCan2b",
    # "Sybil year 1": "sybil_year1",
    # "Sybil year 2": "sybil_year2",
    # "Sybil year 3": "sybil_year3",
    # "Sybil year 4": "sybil_year4",
    # "Sybil year 5": "sybil_year5",
    # "Sybil year 6": "sybil_year6",
}

nlst_preds = data.prep_nlst_preds(nlst_preds_nodule, scanlevel=False, tijmen=True, sybil=False)
print(len(nlst_preds), " nodules")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv")
result_df

3240  nodules


,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.289033,-1.060246,True,0.905856,0.882012,0.930781,False,0.877822,0.855798,0.904268,...,2031,68.179012,8.057945,103,928,31.820988,9.990301,DL_cal,Overweight,demo
de Haas Combined,0.986356,0.017101,True,0.906659,0.890072,0.92301,False,0.907079,0.877049,0.932448,...,2031,68.179012,8.057945,103,928,31.820988,9.990301,Thijmen_mean,Overweight,demo
de Haas Local,0.742882,-0.328039,True,0.905556,0.885175,0.924903,False,0.897261,0.871826,0.921849,...,2031,68.179012,8.057945,103,928,31.820988,9.990301,Thijmen_local_cal,Overweight,demo
de Haas Global (hidden nodule),0.198214,-1.286656,True,0.867675,0.844404,0.888699,False,0.829174,0.800009,0.865511,...,2031,68.179012,8.057945,103,928,31.820988,9.990301,Thijmen_global_hidden_cal,Overweight,demo
de Haas Global (shown nodule),0.08733,-1.709652,True,0.900393,0.879759,0.922863,False,0.852653,0.81602,0.891221,...,2031,68.179012,8.057945,103,928,31.820988,9.990301,Thijmen_global_show_cal,Overweight,demo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
de Haas Combined,0.560479,-0.58213,LightSpeed QX/i,0.916476,0.884431,0.947395,Volume Zoom,0.892479,0.832905,0.933642,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_mean,ManufacturersModelName,scanner
de Haas Local,0.237697,-1.180764,LightSpeed QX/i,0.917535,0.887045,0.947729,Volume Zoom,0.866135,0.794973,0.918767,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_local_cal,ManufacturersModelName,scanner
de Haas Global (hidden nodule),0.668197,-0.428624,LightSpeed QX/i,0.887629,0.858198,0.919354,Volume Zoom,0.868213,0.830912,0.905542,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_global_hidden_cal,ManufacturersModelName,scanner
de Haas Global (shown nodule),0.595375,-0.531063,LightSpeed QX/i,0.922275,0.89482,0.944243,Volume Zoom,0.901086,0.859262,0.937322,...,437,15.308642,11.895161,41,599,19.753086,6.406250,Thijmen_global_show_cal,ManufacturersModelName,scanner


### Scan-level

In [8]:
MODELS = {
    "Venkadesh": "DL_cal",
    # "de Haas Combined": "Thijmen_mean",
    "de Haas Local": "Thijmen_local_cal",
    "de Haas Global (hidden nodule)": "Thijmen_global_hidden_cal",
    "de Haas Global (shown nodule)": "Thijmen_global_show_cal",
    "PanCan2b": "PanCan2b",
    # "Sybil year 1": "sybil_year1",
    # "Sybil year 2": "sybil_year2",
    # "Sybil year 3": "sybil_year3",
    # "Sybil year 4": "sybil_year4",
    # "Sybil year 5": "sybil_year5",
    # "Sybil year 6": "sybil_year6",
}

nlst_preds = data.prep_nlst_preds(nlst_preds_nodule, scanlevel=True, tijmen=False, sybil=True)
print(len(nlst_preds), " scans")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv")
result_df

5911  scans


,p,z,Group_1,AUC_1,AUC-CI-lo_1,AUC-CI-hi_1,Group_2,AUC_2,AUC-CI-lo_2,AUC-CI-hi_2,...,Group_1_ben,Group_1_pct,Group_1_pct_mal,Group_2_mal,Group_2_ben,Group_2_pct,Group_2_pct_mal,col,attribute,category
Venkadesh,0.275465,-1.090564,True,0.894097,0.876698,0.910466,False,0.873489,0.845453,0.895642,...,3640,67.873456,9.272183,209,1690,32.126544,11.005793,DL_cal,Overweight,demo
de Haas Local,0.964593,0.04439,True,0.869771,0.851916,0.887,False,0.870638,0.849109,0.890377,...,3640,67.873456,9.272183,209,1690,32.126544,11.005793,Thijmen_local_cal,Overweight,demo
de Haas Global (hidden nodule),0.641519,-0.465576,True,0.800075,0.781818,0.818452,False,0.789479,0.761269,0.817362,...,3640,67.873456,9.272183,209,1690,32.126544,11.005793,Thijmen_global_hidden_cal,Overweight,demo
de Haas Global (shown nodule),0.026891,-2.213102,True,0.86711,0.851198,0.883208,False,0.82043,0.793541,0.844531,...,3640,67.873456,9.272183,209,1690,32.126544,11.005793,Thijmen_global_show_cal,Overweight,demo
PanCan2b,0.000797,-3.353769,True,0.80803,0.788701,0.83022,False,0.729121,0.695184,0.763285,...,3640,67.873456,9.272183,209,1690,32.126544,11.005793,PanCan2b,Overweight,demo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venkadesh,0.749085,-0.319846,Volume Zoom,0.905674,0.872976,0.939315,Sensation 16,0.896457,0.851294,0.928748,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,DL_cal,ManufacturersModelName,scanner
de Haas Local,0.967376,0.0409,Volume Zoom,0.885999,0.846497,0.924116,Sensation 16,0.887243,0.849386,0.917924,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,Thijmen_local_cal,ManufacturersModelName,scanner
de Haas Global (hidden nodule),0.951275,-0.061105,Volume Zoom,0.805651,0.769375,0.836721,Sensation 16,0.803406,0.767096,0.844879,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,Thijmen_global_hidden_cal,ManufacturersModelName,scanner
de Haas Global (shown nodule),0.488925,-0.69202,Volume Zoom,0.872788,0.837737,0.9042,Sensation 16,0.85009,0.809006,0.882188,...,1204,22.128235,7.951070,96,719,13.787853,11.779141,Thijmen_global_show_cal,ManufacturersModelName,scanner


In [ ]:
MODELS = {
    "Venkadesh": "DL_cal",
    "de Haas Combined": "Thijmen_mean",
    "de Haas Local": "Thijmen_local_cal",
    "de Haas Global (hidden nodule)": "Thijmen_global_hidden_cal",
    "de Haas Global (shown nodule)": "Thijmen_global_show_cal",
    "PanCan2b": "PanCan2b",
    "Sybil year 1": "sybil_year1",
    # "Sybil year 2": "sybil_year2",
    # "Sybil year 3": "sybil_year3",
    # "Sybil year 4": "sybil_year4",
    # "Sybil year 5": "sybil_year5",
    # "Sybil year 6": "sybil_year6",
}

nlst_preds = data.prep_nlst_preds(nlst_preds_nodule, scanlevel=True, tijmen=True, sybil=True)
print(len(nlst_preds), " scans")

result_df = roc.all_results_subgroups_models(nlst_preds, nlst_democols['cat'], models=MODELS, csvpath=f"{RESULTS_DIR}/auroc-nlst-{len(nlst_preds)}.csv", plot=False)
result_df

## Numerical columns